In [17]:

import pandas as pd
import psycopg2
import matplotlib as plt


In [18]:
conn = psycopg2.connect(database="mt_metrics",
                        host="localhost",
                        user="USER",
                        password="PASSWORD",
                        port=5432)
conn.autocommit = True


In [19]:
df = pd.read_sql_query('select t.model_id, m.model_name, m.model_type, t.training_date, t.training_duration, t.epochs, t.learning_rate, t.accuracy, t.loss from training t INNER JOIN models m ON t.model_id=m.model_id', con=conn)

/var/folders/tl/tcgv4w050yj5nx3f9m3gwx6w0000gn/T/ipykernel_12146/2530636970.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query('select t.model_id, m.model_name, m.model_type, t.training_date, t.training_duration, t.epochs, t.learning_rate, t.accuracy, t.loss from training t INNER JOIN models m ON t.model_id=m.model_id', con=conn)


In [20]:
df
# df['accuracy_list'] = df['accuracy_list'].apply(pd.to_numeric, downcast='float')
# df['loss_list'] = df['loss_list'].apply(pd.to_numeric, downcast='float')

KeyError: 'accuracy_list'

In [23]:
print(df.to_markdown())

|    |   model_id | model_name   | model_type            | training_date              |   training_duration |   epochs |   learning_rate |   accuracy |     loss |
|---:|-----------:|:-------------|:----------------------|:---------------------------|--------------------:|---------:|----------------:|-----------:|---------:|
|  0 |          1 | pima2        | binary classification | 2024-01-16 20:08:30.594091 |                   1 |       15 |           0.001 |   0.636719 | 0.691566 |


In [7]:
!pip install tabulate

In [ ]:
df

In [10]:
df.columns[7:9]

Index(['accuracy', 'loss'], dtype='object')

In [ ]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource

# create dict as basis for ColumnDataSource
# data = {'x_values': [1, 2, 3, 4, 5],
#         'y_values': [6, 7, 2, 3, 6]}

# # create ColumnDataSource based on dict
# source = ColumnDataSource(data=data)
source = ColumnDataSource(df)

# create a plot and renderer with ColumnDataSource data
p = figure(height=250)
p.circle(x='x_values', y='y_values', size=20, source=source)
show(p)

In [ ]:
# filter -- model name, model type 
# x -- epochs, learning rate, duration 
# y -- accuracy, loss 


import panel as pn

pn.extension()

x_cols = list(df.columns)[4:7]
y_cols = list(df.columns)[7:9]


x = pn.widgets.Select(name='x', options=x_cols)
y = pn.widgets.Select(name='y', options=y_cols, value='accuracy')
# n_clusters = pn.widgets.IntSlider(name='n_clusters', start=2, end=5, value=3)

# explanation = pn.pane.Markdown(...)

@pn.cache
def get_chart(x, y, df):
    centers = df.groupby('labels')[[x] if x == y else [x, y]].mean()
    return (
        alt.Chart(df)
            .mark_point(size=100)
            .encode(
                x=alt.X(x, scale=alt.Scale(zero=False)),
                y=alt.Y(y, scale=alt.Scale(zero=False)),
                shape='labels',
                color='species'
            ).add_params(brush) +
        alt.Chart(centers)
            .mark_point(size=250, shape='cross', color='black')
            .encode(x=x+':Q', y=y+':Q')
    ).properties(width='container', height='container')

interactive_plot = pn.bind(plot, x, y)

pn.Row(
    pn.WidgetBox(x, y),  
    interactive_plot
)


In [ ]:
import panel as pn
import hvplot.pandas
import pandas as pd
import numpy as np

pn.extension(design='material')

data = df

data.tail()

In [ ]:
### TODO:
# get filter for model name / model type 
# control axes for 

def create_plot():
    return data.hvplot.scatter(x='epochs', y='training_duration', by='model_name', height=400, width=400, legend=False)

create_plot()

In [ ]:
from bokeh.models import Select
from bokeh.layouts import column, row

TOOLTIPS=[
    ("Model Name", "@model_name"),
    ("Model Type", "@model_type")
]
axis_map = {
    "# of Epochs": "epochs",
    "Accuracy": "accuracy",
    "Training Duration": "training_duration",
    "Learning Rate": "learning_rate",
    "Loss": "loss"
}

x_axis = Select(title="X Axis", options=sorted(axis_map.keys()), value="# of Epochs")
y_axis = Select(title="Y Axis", options=sorted(axis_map.keys()), value="Accuracy")
source = ColumnDataSource(
    data=dict(
        x=[], 
        y=[], 
        model_name=[], 
        model_type=[], 
        training_duration=[], 
        epochs=[], 
        learning_rate=[], 
        accuracy=[], 
        loss=[]
    )
)

p = figure(height=600, title="", toolbar_location=None, tooltips=TOOLTIPS, sizing_mode="stretch_width")
p.scatter(x="x", y="y", source=source, size=7, line_color=None)


def update():
    # df = select_movies()
    x_name = axis_map[x_axis.value]
    y_name = axis_map[y_axis.value]

    p.xaxis.axis_label = x_axis.value
    p.yaxis.axis_label = y_axis.value
    p.title.text = f"{len(df)} models selected"
    source.data = dict(
        x=df[x_name],
        y=df[y_name],
        model_name=df["model_name"], 
        model_type=df["model_type"], 
        training_duration=df["training_duration"], 
        epochs=df["epochs"], 
        learning_rate=df["learning_rate"], 
        accuracy=df["accuracy"], 
        loss=df["loss"]
    )

controls = [x_axis, y_axis]
for control in controls:
    control.on_change('value', lambda attr, old, new: update())

inputs = column(*controls, width=320, height=800)
layout = column(row(inputs, p, sizing_mode="inherit"), sizing_mode="stretch_width", height=800)

update()

show(p)

In [ ]:
import seaborn as sns
sns.palplot(sns.color_palette("Set2", 8))

In [ ]:
model_labels = df['model_name'].unique()

In [ ]:
model_labels = df['model_name'].unique()


rgb_values = sns.color_palette("Set2", len(model_labels))
color_map = dict(zip(model_labels, rgb_values))

# # Finally use the mapped values
# plt.scatter(df['epochs'], df['accuracy'], c=df['color'].map(color_map))

In [ ]:
rgb_values

In [ ]:
color_map

In [ ]:
color_map

df['color'] = df['model_name'].apply(lambda x: color_map.get(x))

In [ ]:
df.iloc[0]

In [ ]:
new = [12]
# list(range(1, df.iloc[[new][0]]['epochs'] + 1))
df.iloc[[new][0]]['accuracy_list']

In [24]:
new = [12]
plot = figure()
# print(new[0])
plot.line(x=list(range(1, df.iloc[[new[0]][0]]['epochs'] + 1)), y=df.iloc[[new[0]][0]]['accuracy_list'], legend_label=str('accuracy_list'), color='green')
plot.line(x=list(range(1, df.iloc[[new][0]]['epochs'] + 1)), y=df.iloc[[new][0]]['loss_list'], legend_label=str('loss_list'), color='red')
show(plot)

NameError: name 'figure' is not defined